## Dùng fasttext để train daily, update lại thêm cho model trước đó.

https://www.kaggle.com/code/antonsruberts/sentence-embeddings-centorid-method-vs-doc2vec
https://www.kaggle.com/code/inversion/calculating-stable-diffusion-prompt-embeddings


In [2]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from gensim.models.word2vec import LineSentence
import multiprocessing
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors                                                                            
cores = multiprocessing.cpu_count()
print(cores)

8


In [6]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from gensim.models.word2vec import LineSentence
import multiprocessing
import pandas as pd
cores = multiprocessing.cpu_count()
print(cores)

from regress_adapter import get_data

# get last 2 day and format to '2023-09-19'
date_end = pd.to_datetime('today').strftime('%Y-%m-%d')
# get 2 day before date_end
date_start = (pd.to_datetime(date_end) - pd.DateOffset(days=2)).strftime('%Y-%m-%d')

data = get_data('TPB', date_start, date_end)
print(data.head())
print('new data:' + str(len(data)))

8


/Users/ngocp/Documents/projects/pyml/botapp/algorithm/regress_adapter.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                                 text  close      date_y
0   Ép khách hàng mua bảo hiểm bị phạt 100 triệu đ...  19050  2023-09-19
20  Một cá nhân trở thành cổ đông lớn của SJCSau k...  19050  2023-09-19
19  Ngày 19/09/2023: 10 cổ phiếu nóng dưới góc nh...  19050  2023-09-19
18  Liên tiếp vi phạm công bố thông tin trong 1 nă...  19050  2023-09-19
17  Cổ phiếu AGM sàn liên tiếp 5 phiên do thông ti...  19050  2023-09-19
new data:23


In [16]:
# import word_tokenize
from underthesea import word_tokenize

print(type(data['text']))
t = data['text'].apply(lambda x: word_tokenize(x, format="text"))
print(t.head())

<class 'pandas.core.series.Series'>
0     Ép khách_hàng mua bảo_hiểm bị phạt 100 triệu đ...
20    Một cá_nhân trở_thành cổ_đông lớn của SJCSau k...
19    Ngày 19/09/2023 : 10 cổ_phiếu nóng dưới góc nh...
18    Liên_tiếp vi_phạm công_bố thông_tin trong 1 nă...
17    Cổ_phiếu AGM sàn liên_tiếp 5 phiên do thông_ti...
Name: text, dtype: object


In [20]:
import helper
import numpy as np
from importlib import reload

reload(helper)

def clear_text_vi(texts = []):
    clear_ld = lambda t: helper.NLP(t).get_words_feature()
    t = map(clear_ld, texts)
    return list(t)

sent = clear_text_vi(data['text'])

In [22]:
# FastText load pretrained model 

from gensim.models import FastText

path = './data/fast_new.model'
ft_model = FastText.load(path)

print(ft_model.wv['kinh_tế'])

t = ft_model.wv.most_similar('kinh_tế')
print(t)


[-2.92165041e-01  4.78152722e-01 -1.04250944e+00 -9.89680812e-02
  2.15293452e-01  5.34926243e-02 -3.49078998e-02  1.35056242e-01
  3.74158770e-01 -3.62638861e-01 -2.73760036e-02 -2.20389962e-01
  2.12917641e-01  2.81357408e-01 -1.49783850e-01  3.87781769e-01
 -3.67246240e-01 -4.08129245e-01  5.59060216e-01 -6.49103880e-01
  6.23349920e-02  1.97254598e-01 -3.72478157e-01 -1.97908387e-01
  6.14449680e-02 -2.72229105e-01  2.31160477e-01 -1.17393225e-01
  4.53517705e-01 -1.34577110e-01  2.77455091e-01 -2.31039282e-02
 -2.45647542e-02  2.25192048e-02  2.58990228e-01 -1.73478737e-01
  7.31063948e-04  3.63867767e-02  5.45117743e-02  1.41076520e-01
 -2.67114729e-01 -1.90630928e-01 -3.31837311e-02  4.07149643e-01
 -7.60782287e-02 -3.44041407e-01 -5.05530834e-01 -2.18343750e-01
  8.27507526e-02  2.67352551e-01  6.80616200e-01 -1.00240499e-01
  2.33680215e-02  6.66829152e-03 -1.90532103e-01  5.62540233e-01
 -2.50921607e-01  5.19580722e-01 -1.84901297e-01  3.00185740e-01
  2.94797063e-01  1.98652

In [29]:
print(ft_model.corpus_count)

23


In [30]:
print(len(sent))
print('sent', sent[0])
ft_model.build_vocab(sent, update=True)
ft_model.train(corpus_iterable=sent, total_examples=ft_model.corpus_count, epochs=5)

23
sent ['ép', 'khách_hàng', 'mua', 'bảo_hiểm', 'bị', 'phạt', 'triệu', 'đồng', 'mai_phương', 'maiphuongthanhnien@gmail.com', 'gmt', 'nhân_viên', 'tư_vấn', 'sai', 'ép', 'khách_hàng', 'mua', 'bảo_hiểm', 'sẽ', 'bị', 'xử_phạt', 'mạnh_tay', 'hơn', 'theo', 'quy_định', 'trong', 'dự_thảo', 'mới', 'nhất', 'của', 'bộ', 'tài_chính', 'arfasync', 'push', 'k911czvz', 'tại', 'dự_thảo', 'nghị_định', 'sửa_đổi', 'bổ_sung', 'một_số', 'điều', 'của', 'chính_phủ', 'quy_định', 'về', 'kinh_doanh', 'bảo_hiểm', 'bh', 'xổ_số', 'bộ_tài_chính', 'đề_xuất', 'tăng', 'mức', 'xử_phạt', 'đối_với', 'hành_vi', 'vi_phạm', 'quy_định', 'về', 'triển_khai', 'bh', 'nhân_thọ', 'và', 'bh', 'sức_khỏe', 'lên', 'gấp', 'đôi', 'cụ_thể', 'nâng', 'mức', 'xử_phạt', 'hiện_hành', 'từ', 'triệu', 'đồng', 'lên', 'mức', 'triệu', 'đồng', 'với', 'nhiều', 'hành_vi', 'vi_phạm', 'các', 'hành_vi', 'vi_phạm', 'bị', 'phạt', 'trong', 'khung', 'này', 'như', 'tư_vấn', 'viên', 'không', 'giải_thích', 'rõ_ràng', 'đầy_đủ', 'cho', 'người', 'mua', 'về', 'quyền

(46385, 59090)

In [33]:
print(ft_model.corpus_count)
print(ft_model.wv.most_similar('mai_phương'))

23
[('mai_phương_', 0.967894434928894), ('mai_phương_thuý', 0.9289407730102539), ('mai_phương_thúy', 0.9234344959259033), ('maiphuongthanhnien@gmail.com', 0.88444584608078), ('đối_phương', 0.8210639953613281), ('m.phương', 0.7708355784416199), ('lê_phương_mai', 0.7610512375831604), ('đồng_thanh_xuân', 0.7270358204841614), ('đỗ_hà_phương', 0.7269174456596375), ('ngovutb@gmail.com', 0.7227888703346252)]


In [34]:
ft_model.save('./data/fast_new.model')

Translated into plain English, importance of a term is high when it occurs a lot in a given document and rarely in others

